In [ ]:
import pandas as pd
import random

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/iisc/nlp/Assignment2aDataset (1).txt")

In [ ]:
df.columns=['input','output']

In [ ]:
df

,input,output
0,'9 may 1630','1630-05-09'
1,'15/03/2014','2014-03-15'
2,'mar 16 1675','1675-03-16'
3,'jun 16 1640','1640-06-16'
4,'friday 1791 2 09','1791-09-02'
...,...,...
39994,'december 26 1900','1900-12-26'
39995,'15 may 1828','1828-05-15'
39996,'friday april 18 1851','1851-04-18'
39997,'june 11 2070','2070-06-11'


In [ ]:
!pip install transformers datasets evaluate sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
source_lang = "input"
target_lang = "output"
prefix = "translate input date to Numeric date:"

def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

In [ ]:
df['id'] = range(len(df))

# Define a custom function to format the data
def custom_dataset_format(df):
    formatted_data = {'id': df['id'].tolist(), 'translation': []}

    for row in df.itertuples(index=False):
        translation = {'input': row.input, 'output': row.output}
        formatted_data['translation'].append(translation)

    return formatted_data

formatted_data = custom_dataset_format(df)

# Create a DatasetDict
dataset_dict = DatasetDict({'train': Dataset.from_dict(formatted_data)})

print(dataset_dict)





DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 39999
    })
})


In [ ]:
print(dataset_dict["train"][0])

{'id': 0, 'translation': {'input': "'9 may 1630'", 'output': " '1630-05-09'"}}


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
data_train = dataset_dict["train"].train_test_split(test_size=0.2)

In [ ]:
tokenized_books = data_train.map(preprocess_function, batched=True)

Map:   0%|          | 0/31999 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
tokenized_books

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 31999
    })
    test: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_books["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_books["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer,metrics=['accuracy'])  # No loss argument!

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="my_awesome_opus_books_model",
    tokenizer=tokenizer,
)
callbacks = [metric_callback, push_to_hub_callback]

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

Epoch 1/3
1999/1999 [==============================] - 10147s 5s/step - loss: 0.6273 - accuracy: 0.7301 - val_loss: 0.1901 - val_accuracy: 0.8173
Epoch 2/3
1999/1999 [==============================] - 9978s 5s/step - loss: 0.2495 - accuracy: 0.8088 - val_loss: 0.1163 - val_accuracy: 0.8372
Epoch 3/3
1999/1999 [==============================] - 9979s 5s/step - loss: 0.1701 - accuracy: 0.8266 - val_loss: 0.0752 - val_accuracy: 0.8494
